# 第二回レポート課題
## 1. 2層ニューラルネットワークの学習を試してみましょう。

In [11]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import *


class TwoLayerNet:
    
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size, 
                 weight_init_std = 0.01) -> None:
        '''
        Args:
            input_size      : 入力層のニューロン数
            hidden_size     : 隠れ層のニューロン数
            output_size     : 出力層のニューロン数
        Returns:
            None
        '''
        # 重みの初期化
        self.params = {}
        # 一層目のweight, bias
        self.params["w1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        
        # 2層目のweight, bias
        self.params["w2"] = weight_init_std * np.random.randn(hidden_size, output_size, )
        self.params["b2"] = np.zeros(output_size)

    
    def predict(self, x):
        '''
        recognize
        
        Args:
            x   : 画像データ
        Returns:
            y   : ndarray 
        '''
        w1 = self.params["w1"]
        w2 = self.params["w2"]
        
        b1 = self.params["b1"]
        b2 = self.params["b2"]
        
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        '''
        find the value of the loss function
        
        Args:
            x : input_data
            t : labeled training data
        Returns:
            
        '''
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        '''
        Args:
            x : input_data
            t : labeled training data
        Returns:
        '''
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        '''
        find the gradient for the weight parameter
        
        Args:
            x : input_data
            t : labeled training data
        Returns:
        '''
        loss_w = lambda W: self.loss(x, t)
        
        grads = {}
        # 一層目のweight, biasの勾配
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        # 二層目のweight, biasの勾配
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads

In [12]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['w1'].shape
net.params['b1'].shape
net.params['w2'].shape
net.params['b2'].shape

x = np.random.rand(100, 784)    # ダミーの入力データ
y = net.predict(x)

<class 'type'>
